# scrape dta

- öffne Autoren-Seite aus Author-table
- suche alle Einträge mit "Bellistrik :: [Novelle, Roman, Erzählung, usw. Prosa halt]"
- sammle von gefundenen Einträgen Metadaten
- verbinde Metadaten von Eintrag mit Metadaten aus Author-table
- aus Bellistrikkorpus verschiebe Eintrag nach corpus-folder

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import re

In [2]:
authors_df = pd.read_csv("src/author_table.tsv", encoding="utf8", sep = "\t")

In [3]:
authors_df.dtypes

ID                int64
Voller_Name      object
Nachname         object
Vorname          object
Geburtsjahr       int64
Todesjahr       float64
Nationalität     object
Geschlecht       object
Gutenberg        object
Zeno             object
DTA              object
WikiSource       object
Kolimo+          object
Anmerkung        object
dtype: object

In [4]:
def get_html(website):
    try:
        with urllib.request.urlopen(website) as f:
           html_text = f.read().decode('utf-8')
    except urllib.error.URLError as e:
       print(e.reason)
    return html_text

In [5]:
def get_metadata_DTA(div_container):
    # input is a div-container and we test, if it has Novellen in it
    if ("Novelle\n" in div_container.text) | ("Roman\n" in div_container.text) | ("Märchen\n" in div_container.text) | ("Epik\n" in div_container.text) | ("Prosa\n" in div_container.text) | ("Kinderliteratur\n" in div_container.text):
        tr_soup = div_container.find_all("tr")
        for tr in tr_soup:
            if "Titel:" in tr.text:
                # get title
                titles.append(tr.a.text)
                # get filename
                filenames.append(tr.a.get("href").split("/")[-1])
                # get source
                sources.append(tr.a.get("href"))
            if "Erschienen:" in tr.text:
                # get publication year
                text = tr.text
                pub_year = re.search("\d+", text).group(0)
                pub_years.append(pub_year)
            if "Klassifikation:" in tr.text:
                # get genre
                text = tr.text
                genres.append(text.split("::")[1].replace("\n",""))
                
        authors.append(author)

In [6]:
titles = []
filenames = []
pub_years = []
genres = []
authors = []
sources = []

for website in list(authors_df.dropna(subset=["DTA"])["DTA"]):
    author = authors_df["Voller_Name"].loc[authors_df["DTA"]==website].values[0]  
    html_string = get_html(website)
    soup = BeautifulSoup(html_string, 'html.parser')
    
    div_soup = soup.find_all("div", {"class": "bibl-full"})
    if len(div_soup)==1:
        get_metadata_DTA(div_soup[0])
        
    else:
        for div in div_soup:
            get_metadata_DTA(div)

In [7]:
df = pd.DataFrame()
df["title"]=titles
df["author"]=authors
df["filename"]=filenames
df["pubyear"]=pub_years
df["genre"]=genres
df["sources"] = sources

df

,title,author,filename,pubyear,genre,sources
0,Der tolle Invalide auf dem Fort Ratonneau,Achim von Arnim,arnima_invalide_1818,1818,Novelle,https://www.deutschestextarchiv.de/book/show/a...
1,Brigitta,Adalbert Stifter,stifter_brigitta_1910,1910,Novelle,https://www.deutschestextarchiv.de/book/show/s...
2,Bunte Steine,Adalbert Stifter,stifter_steine01_1853,1853,Novelle,https://www.deutschestextarchiv.de/book/show/s...
3,Bunte Steine,Adalbert Stifter,stifter_steine02_1853,1853,Novelle,https://www.deutschestextarchiv.de/book/show/s...
4,Der Nachsommer,Adalbert Stifter,stifter_nachsommer01_1857,1857,Roman,https://www.deutschestextarchiv.de/book/show/s...
...,...,...,...,...,...,...
277,Die Chronik der Sperlingsgasse,Wilhelm Raabe,raabe_sperlingsgasse_1857,1857,Roman,https://www.deutschestextarchiv.de/book/show/r...
278,Stopfkuchen,Wilhelm Raabe,raabe_stopfkuchen_1891,1891,Roman,https://www.deutschestextarchiv.de/book/show/r...
279,Der Büttnerbauer,Wilhelm von Polenz,polenz_buettnerbauer_1895,1895,Roman,https://www.deutschestextarchiv.de/book/show/p...
280,Phaëthon,Wilhelm Waiblinger,waiblinger_phaeton01_1823,1823,Roman,https://www.deutschestextarchiv.de/book/show/w...


In [9]:
df.to_csv("metadata_DTA.csv", encoding="utf8", sep=";")

In [11]:
list(df["sources"])

['https://www.deutschestextarchiv.de/book/show/arnima_invalide_1818',
 'https://www.deutschestextarchiv.de/book/show/stifter_brigitta_1910',
 'https://www.deutschestextarchiv.de/book/show/stifter_steine01_1853',
 'https://www.deutschestextarchiv.de/book/show/stifter_steine02_1853',
 'https://www.deutschestextarchiv.de/book/show/stifter_nachsommer01_1857',
 'https://www.deutschestextarchiv.de/book/show/stifter_nachsommer02_1857',
 'https://www.deutschestextarchiv.de/book/show/stifter_nachsommer03_1857',
 'https://www.deutschestextarchiv.de/book/show/19_ZZ_2786',
 'https://www.deutschestextarchiv.de/book/show/chamisso_schlemihl_1814',
 'https://www.deutschestextarchiv.de/book/show/19_ZZ_2754',
 'https://www.deutschestextarchiv.de/book/show/Yw_7531_1',
 'https://www.deutschestextarchiv.de/book/show/19_ZZ_2755',
 'https://www.deutschestextarchiv.de/book/show/19_ZZ_2749',
 'https://www.deutschestextarchiv.de/book/show/chamisso_schlemihl_1910',
 'https://www.deutschestextarchiv.de/book/show/